# Análise do risco de inadimplência dos mutuários

Neste projeto prepararemos um relatório para a divisão de empréstimos de um banco. Descobriremos se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Este relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar o empréstimo.

## Iniciação

In [1]:
# Carregando todas as bibliotecas
import pandas as pd

# Carregando os dados
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

# Analisando as informações gerais

## Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

In [2]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Na coluna `education` há diferentes tipos de formatação para a mesma frase, isso implica na contagem. Outro problema é o excesso de informação na coluna `purpose`, dificultando a leitura e a análise.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Existem valores ausentes apenas nas colunas `days_employed` e `total_income`.

In [5]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


A primeira vista parecem ser valores ausentes simétricos, porém não é possível afirmar. Para tal, devemos verificar se as linhas ausentes são correspondentes em ambas as colunas

In [6]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada

df[df['days_employed'].isna() & df['total_income'].isna()]

# Provando que são simétricos pela quantidade de linhas ser a mesma

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusão intermediária**

Pela tabela filtrada poodemos afirmar que os valores ausentes são correspondentes, simétricos. Podemos concluir que ou as pessoas preferiram não responder ou elas não possuem algum trabalho ou trabalham para si mesmas e, portanto, não são empregadas, mas sim empregadoras.

- Outro meio para afirmar os valores ausentes simétricos

In [7]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes
df['total_income'].loc[df['total_income'].isna()]

12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: total_income, Length: 2174, dtype: float64

In [8]:
# Verificando a distribuição
na = df['total_income'].isna().sum()
total = len(df.index)
percentage = na/total
print(f'{percentage: .2%}')

 10.10%


Como os valores ausentes representam um pouco mais que 10% dos dados, a melhor opção é preenchê-los.

**Possíveis motivos para valores ausentes nos dados**

O motivo dos valores simétricos estarem ausentes podem apresentar apenas 2 possibilidades:
- não possuir um trabalho;
- trabalham para si mesmas e, portanto, não são empregadas, mas sim empregadoras.

Podemos discartar a possibilidade de não responder o campo, já que haveriam valores ausentes em outras linhas.

In [9]:
# Verificando a distribuição em todo o conjunto de dados
filtered_data = df.loc[(df['total_income'].isna()) & (df['days_employed'].isna())]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


**Conclusão intermediária**

A distribuição é a mesma, isso significa que as hipoteses postas anteriormente  são plausíveis.

In [10]:
# Verificando outros motivos e padrões que possam levar a valores ausentes

print(filtered_data['education'].value_counts())
print()
print(filtered_data['family_status'].value_counts())

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64


**Conclusão intermediária**

Podemos observar que a maioria são pessoas com baixa escolaridade e casadas.

In [11]:
filtered_data[filtered_data['income_type'] == 'business']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
94,1,NaN,34,bachelor's degree,0,civil partnership,1,F,business,0,NaN,having a wedding
121,0,NaN,29,bachelor's degree,0,married,0,F,business,0,NaN,car
135,0,NaN,27,secondary education,1,married,0,M,business,0,NaN,housing
174,0,NaN,55,bachelor's degree,0,widow / widower,2,F,business,0,NaN,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21391,0,NaN,52,secondary education,1,married,0,F,business,0,NaN,purchase of the house for my family
21407,1,NaN,36,secondary education,1,married,0,F,business,0,NaN,building a real estate
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car


**Conclusões**

Dentre os valores filtrados encontramos em sua maioria pessoas que cursaram apenas até o ensino médio, `secondary education` e mais da metade são casados, `married`. Podemos afirmar, a partir desses dados, que os valores ausentes de maneira geral estão relacionados e são propositais dado o fato que estão desempregados, portanto, não há `days_employed` e `total_income`. Há casos também que provam a segunda hipótese de pessoas que têm o próprio negócio, como vimos acima. 


Dada a natureza dos valores ausentes pode-se concluir que são valores propositais, então a melhor opção é substituir os valores ausentes por 0 dado o que cada valor representa em suas respectivas colunas.

## Transformação de dados

In [12]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [13]:
df['education'] = df['education'].str.lower()

In [14]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [15]:
some_college = 744
total = len(df.index)
percentage = some_college/total
print(f'Quantidade de pertencentes a some_college: {percentage: .2%}')

Quantidade de pertencentes a some_college:  3.46%


In [16]:
# Vamos ver a distribuição de valores na coluna `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

O primeiro valor duvidoso com certeza é `20`. É totalmente plausível casais terem 20 filhos, porém considerando a quantidade e a média dos outros valores é mais provável que seja um erro de digitação. O segundo é `-1`. O mais provavel é que preferiram não responder, mas dependendo da quantidade pode ser apenas outro erro.

In [17]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1

In [18]:
# Verificando a coluna `children` novamente para ter certeza de que está tudo corrigido
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Valores positivos significam  que são aposentados, valores negativos são aqueles que ainda trabalham.

Os valores originais podiam estar em formato de data, por isso os números após a vírgula, eles seriam as horas, minutos e segundos, mas não é possível determinar ao certo.

A pessoa não pode ser menor de 18 anos 

In [19]:
suspicious = df[df['dob_years'] < 18]['dob_years'].count()
total = len(df.index)
percentage = suspicious/total
print(f'{percentage: .2%}')

 0.47%


Retirarei os valores abaixo de 18, já que representam 0,47% dos dados

In [20]:
df = df[df['dob_years'] != -1]
df = df[df['dob_years'] != 0]

In [21]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [22]:
df['family_status'].value_counts()
# não existe nenhum valor problematico na coluna family_status

married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: family_status, dtype: int64

In [23]:
df['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

In [24]:
# é melhor excluir a linha já que é apenas uma e não apresenta padrão de resposta, ou seja, é aleatório

df = df[df['gender'] != 'XNA']

In [25]:
df['gender'].value_counts()

F    14164
M     7259
Name: gender, dtype: int64

In [26]:
df['income_type'].value_counts()

employee                       11064
business                        5064
retiree                         3836
civil servant                   1453
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [27]:
# Verificar duplicatas
df.duplicated().sum()

71

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

In [29]:
# Última verificação se temos duplicatas
df.duplicated().sum()

0

In [30]:
len(df.index)

21352

O banco de dados diminuiu ligeiramente após algumas correções. No início tínhamos 21525 entradas, agora temos 21305, ou seja, descartamos 220 linhas que representa `1,02%` do total.

# Trabalhando com valores ausentes

### Restaurando valores ausentes em "_total_income_"

In [31]:
df.isna().sum()

children               0
days_employed       2093
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2093
purpose                0
dtype: int64

In [32]:
# escreveremos uma função que calcule a categoria de idade
def age_category(age):
    if age < 20:
        return '18-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 65:
        return '50-64'
    return '65+'

In [33]:
# Testando a função
age_category(64)

'50-64'

In [34]:
# Criando coluna nova com base na função
df['age_group'] = df['dob_years'].apply(age_category)

In [35]:
# Verificando os valores na nova coluna
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
21348,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,65+
21349,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39
21350,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


In [36]:
df_without_na = df.dropna()
df_without_na.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-64
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-64
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


In [37]:
#verificando se há valores ausentes
df_without_na.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

In [38]:
df['total_income'].describe()

count     19259.000000
mean      26794.133121
std       16494.347322
min        3306.762000
25%       16494.864000
50%       23200.877000
75%       32536.052000
max      362496.645000
Name: total_income, dtype: float64

In [39]:
# Vendo os valores médios de renda com base nos fatores identificados
tab_income_mean = df_without_na.pivot_table(index='age_group', values='total_income', aggfunc='mean')
tab_income_mean

,total_income
age_group,
18-19,16993.942462
20-29,25570.172966
30-39,28312.479963
40-49,28551.375635
50-64,25313.124503
65+,21542.650450


In [40]:
# Vendo os valores medianos de renda com base nos fatores identificados
tab_income_median = df_without_na.pivot_table(index='age_group', values='total_income', aggfunc='median')
tab_income_median

,total_income
age_group,
18-19,14934.901
20-29,22798.665
30-39,24667.528
40-49,24764.229
50-64,21830.250
65+,18471.391


In [41]:
df_without_na.pivot_table(index='age_group', columns='education', values='total_income', aggfunc='mean')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
age_group,,,,,
18-19,NaN,NaN,NaN,16011.592250,18565.702800
20-29,29395.106109,NaN,27695.271520,23414.838942,25389.341421
30-39,34225.243752,18187.3015,21979.584515,25666.783012,31991.246531
40-49,35687.086166,31771.3210,23618.267667,26193.975154,33703.486528
50-64,32982.335874,41906.9125,18676.032750,23588.439930,28287.081922
65+,28177.463470,15800.3990,17332.906000,20273.532997,26241.578462


In [42]:
df_without_na.pivot_table(index='age_group', columns='education', values='total_income', aggfunc='median')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
age_group,,,,,
18-19,NaN,NaN,NaN,15761.5690,14575.717
20-29,25956.164,NaN,25488.9160,21166.8520,22687.198
30-39,28794.931,18187.3015,19542.3265,22912.9930,28463.439
40-49,30282.333,31771.3210,21511.5635,22973.2580,29323.673
50-64,27429.423,41906.9125,17657.4995,20807.0725,24360.460
65+,23353.511,15800.3990,15354.3830,17943.5555,27938.435


In [43]:
df_without_na.pivot_table(index='gender', values='total_income', aggfunc='median')

,total_income
gender,
F,21469.0015
M,26819.5670


In [44]:
df_without_na.pivot_table(index='gender', values='total_income', aggfunc='mean')

,total_income
gender,
F,24664.752169
M,30905.772981


In [45]:
df_without_na.pivot_table(index='income_type', values='total_income', aggfunc='mean')

,total_income
income_type,
business,32397.307219
civil servant,27361.316126
employee,25824.679592
entrepreneur,79866.103000
paternity / maternity leave,8612.661000
retiree,21939.310393
student,15712.260000
unemployed,21014.360500


In [46]:
df_without_na.pivot_table(index='income_type', values='total_income', aggfunc='median')

,total_income
income_type,
business,27563.0285
civil servant,24083.5065
employee,22815.1035
entrepreneur,79866.1030
paternity / maternity leave,8612.6610
retiree,18969.1490
student,15712.2600
unemployed,21014.3605


Para preencher os valores ausentes em `total_income` usarei a mediana visto que por ser um indicador que depende de emprego, educação e até genêro dependendo dos casos, pode ter muita variação.

In [47]:
def fillna_income_median(row):
    if row['age_group'] == '18-19':
        return float(14934.901)
    if row['age_group'] == '20-29':
        return float(22799.258)
    if row['age_group'] == '30-39':
        return float(24692.450)
    if row['age_group'] == '40-49':
        return float(24768.137)
    if row['age_group'] == '50-64':
        return float(21785.588)
    if row['age_group'] == '65+':
        return float(18434.205)

In [48]:
df['total_income'].isna().sum()

2093

In [49]:
df['total_income'] = df['total_income'].fillna(fillna_income_median)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
21348,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,65+
21349,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.61,property,30-39
21350,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


In [50]:
df['total_income'].isna().sum()

0

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     19259 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  object 
 11  purpose           21352 non-null  object 
 12  age_group         21352 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 2.1+ MB


###  Restaurar valores em "_days_employed_"

In [52]:
# Distribuição de `days_employed` medianos com base nos parâmetros identificados
tab_day_med = df_without_na.pivot_table(index='age_group', values='days_employed', aggfunc='median')
tab_day_med

,days_employed
age_group,
18-19,-724.492610
20-29,-997.796009
30-39,-1579.887451
40-49,-1943.965054
50-64,-336.525580
65+,360304.232308


In [53]:
# Distribuição de `days_employed` médios com base nos parâmetros identificados
tab_day_mean = df_without_na.pivot_table(index='age_group', values='days_employed', aggfunc='mean')
tab_day_mean

,days_employed
age_group,
18-19,-633.678086
20-29,-329.414087
30-39,126.004475
40-49,7061.872832
50-64,164596.297498
65+,312933.276967


Usarei a média para preencher os valores ausentes em `days_employed` porque é um indicador que não depende de variáveis como tipo de emprego, educação e genêro, ou seja, não tem valores atípicos.

In [54]:
def fillna_days_mean(row):
    if row['age_group'] == '18-19':
        return 633.678*-1
    if row['age_group'] == '20-29':
        return 450.039*-1
    if row['age_group'] == '30-39':
        return 131.692
    if row['age_group'] == '40-49':
        return 7009.513
    if row['age_group'] == '50-64':
        return 164696.89
    if row['age_group'] == '65+':
        return 312881.595

In [55]:
df['days_employed'].isna().sum()

2093

In [56]:
df['days_employed'] = df['days_employed'].fillna(fillna_days_mean)

In [57]:
df['days_employed'].isna().sum()

0

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21352 non-null  int64 
 1   days_employed     21352 non-null  object
 2   dob_years         21352 non-null  int64 
 3   education         21352 non-null  object
 4   education_id      21352 non-null  int64 
 5   family_status     21352 non-null  object
 6   family_status_id  21352 non-null  int64 
 7   gender            21352 non-null  object
 8   income_type       21352 non-null  object
 9   debt              21352 non-null  int64 
 10  total_income      21352 non-null  object
 11  purpose           21352 non-null  object
 12  age_group         21352 non-null  object
dtypes: int64(5), object(8)
memory usage: 2.1+ MB


In [59]:
df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce')
df['days_employed'] = pd.to_numeric(df['days_employed'], errors='coerce')

In [60]:
df = df.dropna().reset_index(drop=True)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19259 entries, 0 to 19258
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19259 non-null  int64  
 1   days_employed     19259 non-null  float64
 2   dob_years         19259 non-null  int64  
 3   education         19259 non-null  object 
 4   education_id      19259 non-null  int64  
 5   family_status     19259 non-null  object 
 6   family_status_id  19259 non-null  int64  
 7   gender            19259 non-null  object 
 8   income_type       19259 non-null  object 
 9   debt              19259 non-null  int64  
 10  total_income      19259 non-null  float64
 11  purpose           19259 non-null  object 
 12  age_group         19259 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.9+ MB


## Categorização de dados

In [62]:
# Exibindo os valores dos dados selecionados para categorização
data_category = df[['total_income', 'purpose']]
data_category

,total_income,purpose
0,40620.102,purchase of the house
1,17932.802,car purchase
2,23341.752,purchase of the house
3,42820.568,supplementary education
4,25378.572,to have a wedding
...,...,...
19254,35966.698,housing transactions
19255,24959.969,purchase of a car
19256,14347.610,property
19257,39054.888,buying my own car


In [63]:
# Verificando os valores exclusivos
data_category['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'housing', 'transactions with my real estate', 'cars',
       'to become educated', 'getting an education',
       'second-hand car purchase', 'car', 'wedding ceremony',
       'to get a supplementary education', 'purchase of my own house',
       'real estate transactions', 'getting higher education',
       'to own a car', 'purchase of a car', 'profile education',
       'university education', 'buying property for renting out',
       'to buy a car', 'building a real estate', 'housing renovation',
       'going

In [64]:
def purpose_categorization(reason):
    if 'purchase' in reason or 'buy' in reason or 'car' in reason or 'housing' in reason:
        return 'buy'
    
    elif 'educa' in reason or 'university' in reason:
        return 'education'
    
    elif 'wedding' in reason:
        return 'wedding'
    
    elif 'property' in reason or 'build' in reason:
        return 'build'
    
    elif 'transactions' in reason:
        return 'transactions'

    return 'unknown'

In [65]:
# Criando uma coluna com as categorias e conte os valores para elas
df['purpose_group'] = df['purpose'].apply(purpose_categorization)

In [66]:
# Examinando todos os dados numéricos em sua coluna selecionada para categorização
df['purpose_group'].value_counts()

buy             9576
education       3579
build           2266
wedding         2087
transactions    1751
Name: purpose_group, dtype: int64

In [67]:
def purpose_id(row):
    if row == 'buy':
        return 0
    if row == 'education':
        return 1
    if row == 'build':
        return 2
    if row == 'wedding':
        return 3
    if row == 'transactions':
        return 4

In [68]:
# Criando coluna com categorias
df['purpose_id'] = df['purpose_group'].apply(purpose_id)

In [69]:
print(df['purpose_group'].value_counts())

buy             9576
education       3579
build           2266
wedding         2087
transactions    1751
Name: purpose_group, dtype: int64


In [70]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,purpose_id
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,buy,0
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,buy,0
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,buy,0
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education,1
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-64,wedding,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19254,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49,buy,0
19255,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,65+,buy,0
19256,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39,build,2
19257,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,buy,0


In [71]:
cutoff = [3000, 15000, 23000, 32000, 370000]
labels = [0, 1, 2, 3]
df['income_group'] = pd.cut(df['total_income'], bins = cutoff, labels = labels)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19259 entries, 0 to 19258
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          19259 non-null  int64   
 1   days_employed     19259 non-null  float64 
 2   dob_years         19259 non-null  int64   
 3   education         19259 non-null  object  
 4   education_id      19259 non-null  int64   
 5   family_status     19259 non-null  object  
 6   family_status_id  19259 non-null  int64   
 7   gender            19259 non-null  object  
 8   income_type       19259 non-null  object  
 9   debt              19259 non-null  int64   
 10  total_income      19259 non-null  float64 
 11  purpose           19259 non-null  object  
 12  age_group         19259 non-null  object  
 13  purpose_group     19259 non-null  object  
 14  purpose_id        19259 non-null  int64   
 15  income_group      19259 non-null  category
dtypes: category(1), float6

In [72]:
df['income_group'] = df['income_group'].astype(int)

In [73]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_group            object
purpose_group        object
purpose_id            int64
income_group          int32
dtype: object

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [74]:
income_children_debt = df.pivot_table(index='debt', values='income_group', aggfunc='mean')
print(income_children_debt)

children_total = df.groupby(['debt', 'children'])['children'].agg('count')
children_total = children_total.groupby(level=0).apply(lambda x: x*100 / len(df.index))
children_total.to_frame('default_rate%')

      income_group
debt              
0         1.580131
1         1.534229


default_rate%
debt children               
0    0             60.756010
     1             20.592970
     2              8.925697
     3              1.407134
     4              0.160964
     5              0.041539
1    0              4.917182
     1              2.108105
     2              0.960590
     3              0.114232
     4              0.015577

**Conclusão**

Pela taxa de inadimplência baseada na quantidade de filhos é possível observar que de acordo com o aumento de filhos, quanto mais filhos maior a probabilidade da conta ser paga. Ao mesmo tempo, é notável pela média mais proeminente que os grupos com maior renda devem menos, porém, com uma pequena diferença dos que devem.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [75]:
status_debt = df.pivot_table(index='debt', values='family_status_id', aggfunc='mean')
print(status_debt)

family_total = df.groupby(['debt', 'family_status_id'])['family_status_id'].agg('count')
family_total = family_total.groupby(level=0).apply(lambda x: x*100 / len(df.index))
family_total.to_frame('default_rate%')

      family_status_id
debt                  
0             0.961404
1             1.079335


default_rate%
debt family_status_id               
0    0                     53.253025
     1                     17.545044
     2                      4.185056
     3                      5.181993
     4                     11.719196
1    0                      4.371982
     1                      1.749831
     2                      0.285581
     3                      0.394621
     4                      1.313672

**Conclusão**

Os casados, aqueles em união civil e os solteiros são os que têm as taxas mais baixas de pagamento, respectivamente. Pode-se inferir que os viúvos e divorciados representam uma parcela mínima dos inadimplentes, possivelmente devido à sua maior experiência financeira.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [76]:
income_debt = df.pivot_table(index='debt', values='family_status_id', aggfunc='mean')
print(income_debt)

income_default_rate = df.groupby(['debt', 'income_group'])['income_group'].agg('count')
income_default_rate = income_default_rate.groupby(level=0).apply(lambda x: x*100 / len(df.index))
income_default_rate.to_frame('default_rate%')

      family_status_id
debt                  
0             0.961404
1             1.079335


default_rate%
debt income_group               
0    0                 17.789086
     1                 27.311906
     2                 22.472610
     3                 24.310712
1    0                  1.542136
     1                  2.549457
     2                  2.170414
     3                  1.853679

**Conclusão**

Os casados, aqueles em união civil e os solteiros são os que têm as taxas mais baixas de pagamento, respectivamente. Pode-se inferir que os viúvos e divorciados representam uma parcela mínima dos inadimplentes, possivelmente devido à sua maior experiência financeira. Do mesmo modo, uma renda maior diminui as chances do mesmo.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [77]:
# Conferindo os percentuais de inadimplência para cada finalidade de crédito

purpose_default_rate = df.groupby(['debt', 'purpose_group'])['purpose_group'].agg('count')
purpose_default_rate = purpose_default_rate.groupby(level=0).apply(lambda x: x*100 / len(df.index))
purpose_default_rate.to_frame('default_rate%')

default_rate%
debt purpose_group               
0    build              10.878031
     buy                45.750039
     education          16.864842
     transactions        8.364920
     wedding            10.026481
1    build               0.887897
     buy                 3.972169
     education           1.718677
     transactions        0.726933
     wedding             0.810011

**Conclusão**

A maior concentração de inadimplência ocorre com finalidades do tipo `buy` e em seguida `education`. O mesmo vale para aqueles sem dívidas.


# Conclusão Geral 

Com base nas análises realizadas, podemos concluir que as pessoas inadimplentes com um maior número de filhos demonstram uma tendência a cumprir com suas obrigações financeiras, refletindo em uma taxa de inadimplência mais baixa. Por outro lado, aqueles com menos filhos, principalmente casados ou solteiros, parecem estar mais propensos a acumular dívidas, o que resulta em uma diminuição da pontuação de crédito e um aumento no risco de inadimplência. Além disso, observamos que a taxa de inadimplência aumenta em grupos com renda média, especialmente quando o motivo do empréstimo é categorizado como "compra", o que impacta negativamente ainda mais na pontuação de crédito.
